# Planning observations with `astroplan` - Constraints

In [ ]:
import numpy as np

from astropy.table import QTable
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
import pytz

from astroplan import Observer, FixedTarget

import warnings
warnings.filterwarnings('ignore', category=Warning)

---

# List Comprehensions

List comprehensions provide a concise way to create lists (arrays). Common applications are to make new lists where each element is the result of some operations applied to each member of another sequence.

### For example: Create the list: `[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]`

In [ ]:
squares = []                    # create a blank list

for x in range(10):             # foor loop 0 -> 9
    squares.append(x**2)        # calculate x**2 for each x, add to end of list

squares

### You can do the same thing with:

In [ ]:
squares = [x**2 for x in range(10)]

squares

### You can include `if` statements:

In [ ]:
even_squares = []

for x in range(10):
    
    if (x % 2 == 0):
        even_squares.append(x**2)

even_squares

### You can do the same thing with:

In [ ]:
even_squares = [x**2 for x in range(10) if (x % 2 == 0)]

even_squares

---

# Now to observations

## Let us start with a external list of target objects:

In [ ]:
target_table = QTable.read('ObjectList.csv', format='ascii.csv')

In [ ]:
target_table

In [ ]:
targets = [FixedTarget(coord=SkyCoord(ra = RA*u.hourangle, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC in target_table]

In [ ]:
targets

## Observing Night

### You are the most junior member of the team, so you get stuck observing on New Years Eve

In [ ]:
observe_date = Time("2018-01-01", format='iso')

### But, you get to observe in Hawaii

In [ ]:
my_timezone = pytz.timezone('US/Hawaii')

my_location = Observer.at_site('gemini_north')

In [ ]:
observe_start = my_location.twilight_evening_astronomical(observe_date, which='next')  
observe_end = my_location.twilight_morning_astronomical(observe_date, which='next')

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(observe_start))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(observe_end))
print("Astronomical Evening Twilight starts at {0} local".format(observe_start.to_datetime(my_timezone)))
print("Astronomical Morning Twilight starts at {0} local".format(observe_end.to_datetime(my_timezone)))

In [ ]:
# A complete list of built-in observatories can be found by:

#EarthLocation.get_site_names()

In [ ]:
observing_length = (observe_end - observe_start).to(u.h)

print("You can observe for {0:.1f} tonight".format(observing_length))

In [ ]:
observing_range = [observe_start, observe_end]

## Plot the objects

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from astroplan import time_grid_from_range
from astroplan.plots import plot_sky, plot_airmass

In [ ]:
time_grid = time_grid_from_range(observing_range)

In [ ]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(10,10)

fig.tight_layout()

for my_object in targets:
    ax = plot_sky(my_object, my_location, time_grid)

ax.legend(loc=0,shadow=True);

In [ ]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(10,5)

fig.tight_layout()

for my_object in targets:
    ax = plot_airmass(my_object, my_location, time_grid)

ax.legend(loc=0,shadow=True);

# Observing Constraints

In [ ]:
from astroplan import AltitudeConstraint, AirmassConstraint
from astroplan import observability_table

In [ ]:
constraints = [AltitudeConstraint(20*u.deg, 80*u.deg)]

In [ ]:
observing_table = observability_table(constraints, my_location, targets, time_range=observing_range)

print(observing_table)

## Let us add another constraint

In [ ]:
constraints.append(AirmassConstraint(2))

In [ ]:
observing_table = observability_table(constraints, my_location, targets, time_range=observing_range)

print(observing_table)

## Additional Constraints

`from astroplan import CONSTRAINT`

* `AtNightConstraint()` - Constrain the Sun to be below horizon.
* `MoonIlluminationConstraint(min, max)` - Constrain the fractional illumination of the Moon.
* `MoonSeparationConstraint(min, max)` - Constrain the separation between the Moon and some targets.
* `SunSeparationConstraint(min, max)` - Constrain the separation between the Sun and some targets.

In [ ]:
from astroplan import moon_illumination

In [ ]:
moon_illumination(observe_start)

In [ ]:
from astroplan import MoonSeparationConstraint

In [ ]:
constraints.append(MoonSeparationConstraint(45*u.deg))

In [ ]:
observing_table = observability_table(constraints, my_location, targets, time_range=observing_range)

print(observing_table)

In [ ]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(10,5)

fig.tight_layout()

for i, my_object in enumerate(targets):
    
    if observing_table['ever observable'][i]:
        ax = plot_airmass(my_object, my_location, time_grid)

ax.legend(loc=0,shadow=True);

# Conclusion: New Year's Eve 2018, is not the best night to observe